# AMADEUS CHALLENGE

In [1]:
#name of files
!ls -l ../

total 7755000
drwxrwxr-x 4 dsc dsc       4096 may 28 00:14 amadeus_challenge
-rw-rw-r-- 1 dsc dsc 4244874509 abr  2  2018 bookings.csv
-rw-rw-r-- 1 dsc dsc 3696229366 abr  2  2018 searches.csv


In [2]:
#import libraries
import pandas as pd

In [3]:
#Exploring

## Exercise 1: Count the number of lines in Python for each line

In [4]:
#Read files with only one column to reduce the size of the file
bookings_reader=pd.read_csv("../bookings.csv", delimiter="^", iterator=True, chunksize = 10000, usecols=[0])
searches_reader=pd.read_csv("../searches.csv", delimiter="^", iterator=True, chunksize = 10000, usecols=[0])


In [6]:
def file_lines (file):
    length = 0
    for b_chunk in file:
        length += b_chunk.shape[0]
        print("*", end="")
    return length

bookings_len = file_lines(reader_bookings)
searches_len = file_lines(searches)

print("Total lines of 'bookings.csv': {}".format(bookings_len))
print("Total lines of 'searches.csv': {}".format(searches_len))

In [51]:
#pending.... Check for insosisnte date
#print(bookings_chunk["arr_port"].isnull().sum())

In [52]:
#Clean file
#Bookings_first_chunk = 

## Exercise 2: Top 10 arrivals

In [50]:
#Read file including only "arr_port" and "pax" to calculate top 10 airports
chunks_booking=pd.read_csv("../bookings.csv", names=["arr_port","pax"],delimiter="^", iterator=True, chunksize = 10000 )#names=[arr_port,pax]

finished


In [54]:
#Read all the chuncks aggregate data by airport ("arr_port") adding the bookings passangers ("pax")
pieces = [x.groupby('arr_port')['pax'].agg(['sum']) for x in chunks_booking]

#Concatenate all the pieces
agg = pd.concat(pieces).groupby(level=0).sum()

In [48]:
#Display top 10 by nomber of passangers
agg.iloc[:,0].sort_values(ascending=False).head(10)

arr_port
LHR         88809.0
MCO         70930.0
LAX         70530.0
LAS         69630.0
JFK         66270.0
CDG         64490.0
BKK         59460.0
MIA         58150.0
SFO         58000.0
DXB         55590.0
Name: sum, dtype: float64